In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt
%matplotlib inline

/home/motoofi/miniconda2/envs/mlp/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def get_error_and_accuracy(X,Y,error,sess):
    err = 0
    err += sess.run(error, feed_dict={inputs: X, targets: Y})
    return err

In [3]:
def make_kaggle_submission_file(predictions, modelName):
    ofile  = open('./results/Kaggle_submision_{0}.csv'.format(modelName), "wb")
    writer = csv.writer(ofile)
    writer.writerow(["Id","PRP"])
    for i,score in enumerate(predictions):
        writer.writerow([i,int(round(score[0]))])

    ofile.close()

In [4]:
# Loading data set
data_x_train = pd.read_csv("./X_train.csv")
X_test = pd.read_csv("./X_test.csv")
data_y_train = pd.read_csv("./y_train.csv")
# Splitting data set into training and validation sets. 20% of data set is used for validation.
X_train, X_val, Y_train, Y_val = train_test_split(data_x_train, data_y_train, test_size = 0.2, random_state = 0)

In [5]:
#Feature scaling
# It is not needed to scale Ys because they indicate what class a data point belongs to. They are either 1 or 2.
sc_X = StandardScaler()
X_train_scaled = pd.DataFrame(sc_X.fit_transform(X_train))
X_test_scaled = pd.DataFrame(sc_X.transform(X_test))
X_val_scaled = pd.DataFrame(sc_X.transform(X_val))

sc_Y = StandardScaler()
Y_train_scaled = pd.DataFrame(sc_Y.fit_transform(Y_train['PRP'].reshape(-1,1)))
Y_val_scaled = pd.DataFrame(sc_Y.transform(Y_val['PRP'].reshape(-1,1)))

/home/motoofi/miniconda2/envs/mlp/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/home/motoofi/miniconda2/envs/mlp/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/motoofi/miniconda2/envs/mlp/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.


In [6]:
w = tf.Variable(tf.truncated_normal([6, 1], mean=0.0, stddev=1.0, dtype=tf.float64,seed=0))
b = tf.Variable(tf.zeros(1, dtype = tf.float64))

In [7]:
learning_rate = 0.0001
epochs = 5000
l2=0.2


inputs = tf.placeholder(tf.float64, [None, 6], 'inputs')
targets = tf.placeholder(tf.float64, [None,1], 'targets')
predictions = tf.nn.relu(tf.add(b, tf.matmul(inputs, w)))
error = tf.reduce_mean(tf.square(targets - predictions)+(l2*tf.nn.l2_loss(w)))
#regularizer = tf.nn.l2_loss(weights)
#loss = tf.reduce_mean(loss + beta * regularizer)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [ ]:
errors_tr = list()
errors_va = list()
for e in range(epochs):
    running_error = 0.
    _, batch_error = sess.run([optimizer, error], feed_dict={inputs: X_train_scaled, targets: Y_train_scaled})
    
    running_error += batch_error
    errors_tr.append(running_error)
    print('End of epoch {0}: running error average = {1:.2f}'.format(e + 1, running_error))
    if e % 10 == 0:
        errors_va.append(get_error_and_accuracy(X_val_scaled,Y_val_scaled,error,sess))
        print('Valid data: Error={0:.2f}'.format(errors_va[-1]))
        
pre = sess.run(predictions, feed_dict={inputs: X_test_scaled})


In [ ]:
plt.plot( range(0,epochs),errors_tr, 'r--',label='tr')
plt.plot(range(0,epochs,10),errors_va, 'b--',label='va')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
inversed_predictions = sc_Y.inverse_transform(pre)
inversed_predictions